In [3]:
# import LLM 
from langchain_openai import ChatOpenAI
# guide response using template 
from langchain_core.prompts import ChatPromptTemplate
# let's work with strings in the output
from langchain_core.output_parsers import StrOutputParser
# import embedding models
from langchain_openai import OpenAIEmbeddings
# import pdf reader
from langchain_community.document_loaders import PyPDFLoader
# import vectorstore 
from langchain_community.vectorstores import FAISS
# import multiple document text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# import document template for answer retrieval 
from langchain.chains.combine_documents import create_stuff_documents_chain
# retrieval chain helper
from langchain.chains import create_retrieval_chain

In [4]:
import os
from dotenv import load_dotenv # used to load env variables 
load_dotenv()


False

In [1]:
# Retrieval Augmented Generation

class medical_chatbot():
    def __init__(self):
            # load llm
        self.llm = ChatOpenAI() 
        self.n_docs_retrieve = 10
        self.embeddings = OpenAIEmbeddings()
        
        self.__load_docs()
        self.__set_prompt()
        
    def __load_docs(self):
        docs = []
        for doc in os.listdir(os.getenv("PDF_FOLDER_PATH")):
            if doc.endswith('.pdf'):
                pdf_path = os.path.join(os.getenv("PDF_FOLDER_PATH"), doc)
                loader = PyPDFLoader(pdf_path)
                docs.extend(loader.load_and_split())
        
        self.faiss_index = FAISS.from_documents(docs, self.embeddings) 
        
    def __set_prompt(self):
        # set up the chain that takes a question and the retrieved documents and generates an answer.
        self.prompt = ChatPromptTemplate.from_template("""Your role is to provide information regarding melanoma. 
        Attempt to answer the following question based only on the provided context:

            <context>
            {context}
            </context>
            
        If the context doesn't provide an answer, provide an answer from your own knowledge.
        Question: {input}""")
        

    def get_retrieval_augmented_answer(self, query):
        
        docs = self.faiss_index.similarity_search(query, k=self.n_docs_retrieve) # find docs sim to prompt

        text_splitter = RecursiveCharacterTextSplitter()
        documents = text_splitter.split_documents(docs)
        vector = FAISS.from_documents(documents, self.embeddings) # we have this data indexed in a vectorstore
        

        document_chain = create_stuff_documents_chain(self.llm, self.prompt)
        
        # create retrieval 
        retriever = vector.as_retriever() 
        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        
        # get response for prompt
        self.response = retrieval_chain.invoke({"input": query})
        return self.response["answer"]

In [2]:
chatbot = medical_chatbot()

NameError: name 'ChatOpenAI' is not defined

In [22]:

query = 'What is melanoma?'
chatbot.get_retrieval_augmented_answer(query)

"Melanoma is a type of skin cancer that develops in the skin cells called melanocytes. It is considered the most serious form of skin cancer because it is more likely to spread to other parts of the body, especially if not found early. Melanoma most often develops in areas that have been exposed to the sun, but it can also start in areas that don't receive much sun, such as the eye, nasal passages, mouth, genitals, soles of the feet, palms of the hands, and under the nails."

In [23]:
query = 'What is nodular melanoma?' 
chatbot.get_retrieval_augmented_answer(query)

'Nodular melanoma is a type of melanoma of the skin that often grows quickly. It is categorized by its clinical and pathological appearance and is one of the less common types of melanoma. Unlike other types of melanoma that may follow the ABCD signs, nodular melanoma may not fit these guidelines. It is important to be aware of the signs and symptoms of nodular melanoma, such as a rapidly growing raised bump on the skin, and to seek medical attention if any suspicious changes are noticed.'

In [24]:
query = 'What is the probability of death of early detection of melanoma?' 
chatbot.get_retrieval_augmented_answer(query)

'The probability of death from early detection of melanoma is significantly lower compared to cases where melanoma is detected at a later stage. Early detection allows for prompt treatment and a higher chance of successful outcomes. In general, the prognosis for melanoma is more favorable when it is caught and treated in its early stages before it has spread to deeper layers of the skin or other organs. It is important to regularly check your skin for any changes and to seek medical attention if you notice any suspicious moles or spots.'